In [ ]:
!pip install tensorflow-gpu==2.0.0-rc1

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model

print(tf.__version__)

In [ ]:
# importing data from Kaggle

!pip install -q kaggle
from google.colab import files

files.upload()

In [ ]:
# downloading data 

!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json
  
!kaggle competitions download -c image-classification2


In [ ]:
# extracting data 

!tar xvzf imagecl.tar.gz



In [ ]:

train_dir = 'imagecl/train'
test_dir = 'imagecl/test'
for dir in os.listdir(train_dir):
    print(dir)

In [ ]:
image_height = 224
image_width = 224

batch_size = 32
classes_num = 3



In [ ]:
# dataset preparing
#train_data_gen = ImageDataGenerator(rescale = 1. / 255)



train_generator = ImageDataGenerator(rescale = 1. / 255, 
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode = 'nearest',
                                    validation_split=0.2)

train_data = train_generator.flow_from_directory(
    train_dir, 
    subset='training',
    target_size = (image_height, image_width), 
    batch_size = batch_size, 
    class_mode = 'categorical', 
    shuffle = False

)

val_data = train_generator.flow_from_directory(
    train_dir,
    subset='validation',
    target_size = (image_height, image_width), 
    batch_size = batch_size, 
    class_mode = 'categorical', 
    shuffle = False
)

In [ ]:
# model construction


pretrained_model = tf.keras.applications.VGG16(input_shape = (image_height, image_height, 3), include_top = False, weights = 'imagenet')



for layer in pretrained_model.layers[:-4]:
    layer.trainable = False


for layer in pretrained_model.layers:
    print(layer, layer.trainable)

model = tf.keras.models.Sequential()

model.add(pretrained_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(classes_num, activation = 'softmax'))




model.summary()








In [ ]:
# compile model 


optimze_algo = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(optimizer=optimze_algo, loss = 'categorical_crossentropy',  metrics = ['acc'])


In [ ]:
accuracy_threshold = 0.99
patience = 50


class callback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('acc') > accuracy_threshold):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(accuracy_threshold*100))
            self.model.stop_training = True
            self.model.save('model-optimal.h5')
        

        
checkpoint = tf.keras.callbacks.ModelCheckpoint('weights{epoch:02d}.h5', save_weights_only=True, save_freq=10)     
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau('val_acc', factor=0.2, patience=7, verbose=1)
callbacks = [callback(), checkpoint]







In [ ]:
epochs_num = 90

history = model.fit_generator(train_generator,
            validation_data = val_generator,
            steps_per_epoch = 25,
            epochs = epochs_num,
            validation_steps = 6, 
            shuffle = True, 
            verbose = 2, callbacks=callbacks)
            

In [ ]:
model_name =  'model-991-885.h5'

model.save(model_name)

#loaded_model = tf.keras.models.load_model(model_name)



In [ ]:
#mounting google drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# saving model to google drive

path = '/content/gdrive/My Drive/DL-models'
path = os.path.join(path, model_name)

model.save(path)



In [ ]:
loaded_model = tf.keras.models.load_model(path)

In [ ]:
# examining model 

import matplotlib.pyplot as plt



acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
import numpy as np 

img_dir= test_dir
img_list = os.listdir(img_dir)

batch_holder = np.zeros((50, image_height, image_width, 3))
for i,img in enumerate(img_list[10: 20]):
    img = tf.keras.preprocessing.image.load_img(os.path.join(img_dir,img), target_size=(image_height,image_height))
    batch_holder[i, :] = img
  
  

labels = train_generator.class_indices

labels = {v: k for k, v in labels.items()}

print(labels)



result = loaded_model.predict_classes(batch_holder)
 
fig = plt.figure(figsize=(20, 20))
 
for i,img in enumerate(batch_holder):
    fig.add_subplot(4,5, i+1)
    plt.title(labels[result[i]])
    plt.axis('off')
    plt.imshow(img/256.) 

plt.show()

In [ ]:
# testing on uploaded images

from google.colab import files

image = files.upload()
image = list(image.keys())[0]
img = tf.keras.preprocessing.image.load_img(image, target_size=(image_height,image_width))
img = np.array(img)
img = img.reshape(1, image_height,image_width, 3)
pred = np.argmax(loaded_model.predict(img))

print(labels[pred])





